In [1]:
#On réunira ici toutes les déclarations de variables globales
#On importera également nos bibliothèques ici

import numpy as np
import random as rd
import copy as copy
import uuid as uuid

#type(n) : int
#n est la taille de la matrice carrée
n = 100

#type(probabilite_presence_carotte) : float [0.00 ; 1.00]
#probabilite_presence_carotte est la probabilité qu'il n'y ait pas de carottes sur une case
probabilite_presence_carotte = 0.5

#type(esperance_vie_renard) : int
#esperance_vie_renard est un nb de jours
esperance_vie_renard = 1825 #environ 5 ans

#type(esperance_vie_lapin) : int
#esperance_vie_lapin est un nb de jours
esperance_vie_lapin = 2555 #environ 7 ans

#type(efficacite_renard) : float
#efficacite_renard est l'efficacité de chasse du renard, i.e. sa chance d'attraper le lapin
efficacite_renard_generale = 0.20 #basé sur des données réelles
def efficacite_renard_specifique():
    return rd.uniform(-0.1,0.1)

#type(efficacite_lapin) : float
#efficacite_lapin est un pondérateur fictif pour éviter un système mathématique trop prédominant, c'est le facteur chance naturel
def efficacite_lapin_specifique():
    return rd.uniform(-0.1,0.1)

#type(nb_renards_initial) : int
#nb_renards_initial le nombre initial de renards
nb_renards_initial = 500

#type(nb_lapins_initial) : int
#nb_lapins_initital le nombre initial de lapins
nb_lapins_initial = 1000

#type(M) : int
#M le range de l'entourage de Moore
M = 1

#type(maturite_sexuelle_renard) : int
#maturite_sexuelle_renard le nb de jours pour pouvoir s'accoupler
maturite_sexuelle_renard = 310 #donnée réelle, correspond à 10 mois

#type(temps_gestation_renard) : int
#temps_gestation_renard le nb de jours avant de mettre bas
temps_gestation_renard = 55 #donnée réelle

# Agents

### Renard

In [2]:
def creation_renard():
    ID = uuid.uuid4()
    age = rd.randint(0,esperance_vie_renard)
    sexe = rd.choice([True, False]) #True correspond à femelle
    efficacite = efficacite_renard_generale + efficacite_renard_specifique()
    fecondite = rd.uniform(0.2,1.0)
    position = (rd.randint(0,n-1),rd.randint(0,n-1))
    faim = rd.random()
    excitation = rd.random()
    gestationnaire = (False, 0, 0) #en gestation ou non, nb de jour avant mis à bas, nb de renardeaux
    type_agent = "renard"
    
    return (ID, (type_agent, (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position)))

In [3]:
print(creation_renard())

(UUID('d47cf862-29aa-448e-8d15-eacc9e7b24ab'), ('renard', (1464, False, 0.10545725909593061, 0.8701387635555975, 0.03408680698376754, 0.6495262051226595, (False, 0, 0), (41, 17))))


In [4]:
def creation_renards():
    cpt = 0 #type(cpt) : int
    renards = dict()
    while cpt < nb_renards_initial:
        cpt += 1
        identifiant, renard = creation_renard()        
        renards[identifiant] = renard
    return renards

In [5]:
print(creation_renards())

{UUID('984caaf0-09a1-4283-ad44-1d3ddd71dbf9'): ('renard', (155, False, 0.16920915833514286, 0.9194728497043851, 0.31508584197411726, 0.5519254814983816, (False, 0, 0), (30, 95))), UUID('20c7a949-f58d-4928-bf27-e3167adb7a7b'): ('renard', (788, False, 0.20702424718712195, 0.26655155348714854, 0.6420264018739168, 0.14985408507497788, (False, 0, 0), (41, 15))), UUID('142de4cd-3933-4695-aba3-2983eef89697'): ('renard', (1361, False, 0.18450367132814782, 0.5261152834362673, 0.44052407700669227, 0.1003361765635582, (False, 0, 0), (92, 99))), UUID('011e2084-76d1-4242-a31d-705fbcd07ce5'): ('renard', (1556, False, 0.27966880572459446, 0.3970140728099909, 0.8167893134946566, 0.5510325210236923, (False, 0, 0), (57, 11))), UUID('f17d0c84-024b-435f-8df3-3480b300b3b5'): ('renard', (354, False, 0.20128421288733536, 0.4293683677408655, 0.022365007817351734, 0.11360008782075237, (False, 0, 0), (4, 63))), UUID('b5d0d13a-88c5-4739-9191-5380050e1c28'): ('renard', (10, False, 0.11854481006817899, 0.376288213

### Lapin

In [6]:
def creation_lapin():
    """None -> lapin
    lapin : tuple[int, bool, float, float, float, float, tuple[bool, int], tuple[int, int]]"""
    ID = uuid.uuid4()
    age = rd.randint(0, esperance_vie_lapin)
    sexe = rd.choice([True, False]) #True correspond à femelle
    efficacite = efficacite_lapin_specifique()
    fecondite = rd.uniform(0.2,1.0)
    position = (rd.randint(0,n-1),rd.randint(0,n-1))
    faim = rd.random()
    excitation = rd.random()
    gestationnaire = (False, 0, 0) #en gestation ou non, nb de jour avant mis à bas, nb de lapinous
    type_agent = "lapin"
    
    return (ID, (type_agent, (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position)))

In [7]:
print(creation_lapin())

(UUID('e5c3c229-9eaf-425d-93c6-1f490db4f81a'), ('lapin', (1233, True, -0.021535095203237903, 0.8469529144493755, 0.30628259646927103, 0.0959457682488224, (False, 0, 0), (39, 69))))


In [8]:
def creation_lapins():
    cpt = 0 #type(cpt) : int
    lapins = dict()
    while cpt < nb_renards_initial:
        cpt += 1
        identifiant, lapin = creation_lapin()        
        lapins[identifiant] = lapin
    return lapins

In [9]:
print(creation_lapins())

{UUID('08d43ef8-8689-4675-b600-e5a29a5084f1'): ('lapin', (734, True, 0.041082777906549794, 0.5385218020981568, 0.49447810568361783, 0.37121768291875545, (False, 0, 0), (77, 75))), UUID('e36a9c65-5b8a-49cb-943f-480ba2b359c7'): ('lapin', (2249, True, 0.08268129612511041, 0.657614102552178, 0.017042900167651864, 0.9835904675502292, (False, 0, 0), (83, 88))), UUID('efac9975-2f65-4032-8eda-9de6d7784076'): ('lapin', (2363, True, -0.06057215220657302, 0.46223668504135706, 0.12677106483030598, 0.2084684106585416, (False, 0, 0), (48, 9))), UUID('9bd71f9d-1fad-49d8-bc8d-217827313bcc'): ('lapin', (65, True, 0.01618693343953026, 0.9937512444715548, 0.4890101544929619, 0.21167105797626584, (False, 0, 0), (59, 45))), UUID('c6c0b52d-439c-411f-bb6a-c860c9f2215b'): ('lapin', (117, False, -0.06569733683972634, 0.5629513008575433, 0.4465425531940139, 0.5970176234666964, (False, 0, 0), (67, 2))), UUID('03ce986c-982a-4ad1-bbd9-b89a847a5c39'): ('lapin', (218, False, -0.06283699465399291, 0.49135394547561156

### Carotte

In [10]:
def creation_carotte(ligne, colonne):
    return (uuid.uuid4(), ("carotte", (rd.random(),(ligne, colonne))))

In [11]:
print(creation_carotte(0,0))

(UUID('7fd1be62-9e74-42a9-b27f-53701e65f7ed'), ('carotte', (0.3680876970377367, (0, 0))))


In [12]:
def creation_carottes():
    cpt = 0
    carottes = dict()
    while cpt < n*n:
        cpt += 1
        if rd.random() >= probabilite_presence_carotte:
            identifiant, carotte = creation_carotte((cpt-(cpt%100))//100,cpt%100)
            carottes[identifiant] = carotte
    return carottes

In [13]:
print(creation_carottes())

{UUID('b74a8b3c-0ff7-44ec-a670-bbafc03df427'): ('carotte', (0.6264878124014145, (91, 26))), UUID('fdef9d47-66a0-41c4-a050-678d8eb9d967'): ('carotte', (0.4459678190648769, (42, 62))), UUID('4342983d-a110-4e2e-8127-5aecb1f2dea9'): ('carotte', (0.15457345132513767, (15, 50))), UUID('e2627c15-4004-4dcf-86ef-046b311531b3'): ('carotte', (0.7289796184446747, (33, 50))), UUID('9141d727-1740-40ae-a30c-c1448c2b06ea'): ('carotte', (0.02624830188583327, (40, 37))), UUID('d1f03f2d-b251-4c61-8651-2009242229bd'): ('carotte', (0.49957965043072505, (39, 91))), UUID('37c75051-97f6-490b-8b79-91c1d19034e7'): ('carotte', (0.24224271573937473, (13, 18))), UUID('3f6e61e8-ef2c-4554-8d74-ba89fc6a3e41'): ('carotte', (0.998744215826295, (72, 34))), UUID('da228e2f-7efa-41cd-a945-be255c0f3e55'): ('carotte', (0.9643380109390002, (85, 98))), UUID('7b7cb129-1b8e-431c-aabc-7b24b567f0a3'): ('carotte', (0.5701371450622683, (49, 23))), UUID('7d8001ea-c6b0-44c4-adfd-1dc9c9ba5b95'): ('carotte', (0.21763259098354903, (59, 9

### Mélange des agents

On déclare en variable globale le dictionnaire rassemblant tous nos agents

In [14]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [15]:
agents = merge_two_dicts(creation_lapins(), creation_renards())
carottes = creation_carottes()

In [16]:
print(agents)
print(carottes)

{UUID('9928ad87-6458-44eb-8172-76daa55946b3'): ('renard', (643, True, 0.2612855720605586, 0.7769298549108508, 0.2087692625243689, 0.7908676149299828, (False, 0, 0), (73, 34))), UUID('931987c3-d8c1-4992-ae50-d070b805463d'): ('lapin', (196, True, -0.07985345819312897, 0.405857705612755, 0.8540429101820842, 0.620147521872345, (False, 0, 0), (20, 52))), UUID('64237e23-ca71-4819-9506-6ef53965fa38'): ('renard', (51, True, 0.2666360360467987, 0.8581164069015108, 0.03442513591344021, 0.6540155996993403, (False, 0, 0), (53, 27))), UUID('80840678-dbb1-49ba-b503-dd7dc60f8255'): ('lapin', (1484, False, -0.06334085551970398, 0.8419726460122889, 0.5167003947792445, 0.04980929030049064, (False, 0, 0), (73, 18))), UUID('716a4eb2-ddde-401e-bf97-367c13f2a809'): ('renard', (320, False, 0.22834311311520816, 0.425588115414488, 0.4123110270850737, 0.050196083946910885, (False, 0, 0), (41, 23))), UUID('c0121a0a-8e18-454d-89bb-c002e6f9e883'): ('renard', (651, True, 0.10315040618461949, 0.387221952070383, 0.57

# Environnement

### Initialisation de l'environnement

In [17]:
def init_matrice():
    a = np.empty((n,n), dtype=np.object_)
    a.fill([])
    a = np.frompyfunc(list,1,1)(a)
    return a

In [18]:
environnement = init_matrice()

In [19]:
print(environnement)

[[[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 ..., 
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]]


In [20]:
test = init_matrice()
test[0,1].append("banane")
print(test)

[[[] ['banane'] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 ..., 
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]]


### Placement des agents dans l'environnement

In [21]:
def update_matrice():
    for e in agents:
        _,agent = agents[e]
        _,_,_,_,_,_,_,position = agent
        ligne, colonne = position
        environnement[ligne, colonne].append(e)
    return environnement

In [22]:
print(update_matrice())

[[[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [UUID('bc52ab3b-33bd-4671-9336-894e49bdf174')] []]
 ..., 
 [[] [] [UUID('b6c967cc-29ae-421e-a434-fb05a5fa97d1')] ..., [] [] []]
 [[] [] [] ..., [] [] []]
 [[] [] [] ..., [] [] []]]


In [23]:
def uptade_matrice_carotte():
    for e in carottes:
        _,carotte = carottes[e]
        _,position = carotte
        ligne, colonne = position 
        environnement[ligne, colonne].append(e)
    return environnement

In [24]:
print(uptade_matrice_carotte())

[[[] [UUID('a7374939-42a7-4e76-bcd9-4af6a8a87199')]
  [UUID('224d9b3b-4de5-42ef-a832-034e0a6c8c4e')] ...,
  [UUID('79928ed9-f4c8-4b83-95b4-7f422321a32c')] []
  [UUID('974bd3d2-e6c6-44de-bc5b-dcb302233daf')]]
 [[UUID('4cf8176f-08e1-4def-9079-08e88b6e0197')] [] [] ..., []
  [UUID('fe0f4022-df52-4015-917d-28e0b9d686a1')]
  [UUID('0126a677-5521-4649-ba2b-7fd0a9da436d')]]
 [[UUID('0b9fccbc-15ef-496b-a63d-6e46522e9428')] []
  [UUID('75a5b995-b43b-4909-b162-a876301e36e9')] ...,
  [UUID('523dd391-2793-467c-a634-893680f63473')]
  [UUID('bc52ab3b-33bd-4671-9336-894e49bdf174'), UUID('f8f71403-fb90-4a3b-b9bc-6d95957029a3')]
  []]
 ..., 
 [[] []
  [UUID('b6c967cc-29ae-421e-a434-fb05a5fa97d1'), UUID('4cc943d2-1186-4074-a950-b38cd4010a1f')]
  ..., [] [UUID('0db83bfc-b538-43e0-9818-39fa3fa45e9c')] []]
 [[UUID('1e9fb34b-d26c-49c5-893a-5f7d6dd10ddc')]
  [UUID('8999bc27-d969-43ef-b9b0-382ebf972285')]
  [UUID('e2482939-ba00-4b8e-9ea9-4bf31a67eee6')] ...,
  [UUID('5d4edd94-d2a4-4bd5-af50-51913ecf6a9f')]
  

### Interaction des agents

In [5]:
def reussite_chasse(efficacite_renard,efficacite_lapin):
    """donne un booléen qui correspond à l'issue de la chasse du renard"""
    return rd.random() < (efficacite_renard-efficacite_lapin)

In [4]:
def nb_renardeaux(age1,age2,fec1,fec2):
    """donne le nombre de renardeaux dans une portée aléatoirement"""
    #d : int
    d = 0
    if age1 > maturite_sexuelle_renard and age2 > maturite_sexuelle_renard:
        if (fec1 + fec2) / 2 >= 0.5:
            d = rd.choice[4,5]
    return d

In [52]:
def mouvement_renard(id_renard):
    
    _, renard = agents(id_renard)
    age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position = renard
    ligne, colonne = position
    en_gestation, jour, nb = gestationnaire
    
    entourage_moore = [] #liste des uuid aux environs
    
    for a in range(ligne-M,ligne+M):
        for b in range(colonne-M,colonne+M):
            for c in environnement[a,b]:
                entourage_moore.append(c)
    
    #On retire les carottes de l'entourage
    entourage_moore_renard = []
    entourage_moore_lapin = []
    
    for e in entourage_moore:
        type_agent,_ = agents[e]
        if type_agent == "renard":
            entourage_moore_renard.append(e)
        elif type_agent == "lapin":
            entourage_moore_lapin.append(e)        
    
    #On examine si le renard veut manger ou se reproduire
    if (faim >= excitation and len(entourage_moore_lapin) != 0) or (faim < excitation and len(entourage_moore_lapin) != 0 and len(entourage_moore_renard)==0):
        
        id_proie = rd.choice(entourage_moore_lapin)
        _,proie = agents[id_proie]
        _,_,efficacite_proie,_,_,_,_, position_proie = proie
        ligne_proie, colonne_proie = position_proie
        
        if reussite_chasse(efficacite, efficacite_proie):
            environnement[ligne_proie, colonne_proie].remove(id_proie)
            faim = 0
        #On repack le tuple du renard
        position = (ligne_proie, colonne_proie)
        agents[id_renard] = ("renard", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))
        environnement[ligne, colonne].remove(id_renard)
        environnement[ligne_proie, colonne_proie].append(id_renard)
        
    elif (faim < excitation and len(entourage_moore_renard) != 0) or (faim >= excitation and len(entourage_moore_lapin) == 0 and len(entourage_moore_renard) !=0):
        
        id_proie = rd.choice(entourage_moore_renard)
        _,proie = agents[id_proie]
        age_proie, sexe_proie,_, fecondite_proie,_,_,gestationnaire_proie,position_proie = proie
        en_gestation_proie, jour_proie, nb_proie = gestationnaire_proie
        ligne_proie, colonne_proie = position_proie
        
        #Les deux sexes doivent être différents.
        #Aucun des deux ne doit être en gestation, et le male n'est jamais en gestation.
        if sexe_proie != sexe and en_gestation == en_gestation_proie:
            excitation = 0
            excitation_proie = 0
            if sexe:
                nb = nb_renardeaux(age, age_proie, fecondite, fecondite_proie)
                jour = temps_gestation_renard
                en_gestation = True
                gestationnaire = (en_gestation, jour, nb)
                position = (ligne_proie, colonne_proie)
            else:
                nb_proie = nb_renardeaux(age, age_proie, fecondite, fecondite_proie)
                jour_proie = temps_gestation_renard
                en_gestation_proie = True
                gestationnaire_proie = (en_gestation_proie, jour_proie, nb_proie)
                position = (ligne_proie, colonne_proie)
            agents[id_renard] = ("renard", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))
            agents[id_proie] = ("renard", (age_proie, sexe_proie, efficacite_proie, fecondite_proie, excitation_proie, faim_proie, gestationnaire_proie, position_proie))
        environnement[ligne, colonne].remove(id_renard)
        environnement[ligne_proie, colonne_proie].append(id_renard)
    else:
        #Mouvement aléatoire
        ligne += rd.choice(range(-M,M+1))
        colonne += rd.choice(range(-M,M+1))
        position = (ligne, colonne)
        agents[id_renard] = ("renard", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))

In [1]:
def nb_lapinous(age1,age2,fec1,fec2):
    """donne le nombre de lapinous dans une portée en fonction des fécondités des parents"""
    #k:int
    k=0
    if age1>124 and age2>124:
        if (fec1+fec2)/2>=0.2:
            k=4+int(((fec1+fec2)/2-0.2)*10)
    return k

In [3]:
def mouvement_lapin(id_lapin):
    
    _, lapin = agents(id_lapin)
    age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position = lapin
    ligne, colonne = position
    en_gestation, jour, nb = gestationnaire
    
    entourage_moore = [] #liste des uuid aux environs
    
    for a in range(ligne-M,ligne+M):
        for b in range(colonne-M,colonne+M):
            for c in environnement[a,b]:
                entourage_moore.append(c)
    
    #On retire les renards de l'entourage
    #Cela implique qu'il n'y a pas d'interactions renard-lapin venant du lapin. Pas de fuite (?)
    entourage_moore_carotte = []
    entourage_moore_lapin = []
    
    for e in entourage_moore:
        type_agent,_ = agents[e]
        if type_agent == "carotte":
            entourage_moore_carotte.append(e)
        elif type_agent == "lapin":
            entourage_moore_lapin.append(e)
    
    #On examine si le lapin veut manger ou se reproduire
    if (faim >= excitation and len(entourage_moore_carotte) != 0) or (faim < excitation and len(entourage_moore_lapin) == 0 and len(entourage_moore_carotte)!=0):
        
        #Cas de la prédation
        
        id_proie = rd.choice(entourage_moore_carotte)
        _, proie = agents[id_proie]
        nb_proie, position_proie = proie
        ligne_proie, colonne_proie = position_proie
        
        if nb_proie >= .1:
            nb_proie -= 0.1
            faim = 0
        #On repack le tuple du lapin et de la carotte
        position = (ligne_proie, colonne_proie)
        agents[id_lapin] = ("lapin", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))
        agents[id_proie] = ["carotte"], (nb_proie, position_proie)
        environnement[ligne, colonne].remove(id_lapin)
        environnement[ligne_proie, colonne_proie].append(id_lapin)
        
    elif (faim < excitation and len(entourage_moore_lapin) != 0) or (faim >= excitation and len(entourage_moore_carotte) == 0 and len(entourage_moore_lapin) !=0):
        
        #Cas de la reproduction
        
        id_proie = rd.choice(entourage_moore_lapin)
        _,proie = agents[id_proie]
        age_proie, sexe_proie,_, fecondite_proie,_,_,gestationnaire_proie,position_proie = proie
        en_gestation_proie, jour_proie, nb_proie = gestationnaire_proie
        ligne_proie, colonne_proie = position_proie
        
        #Les deux sexes doivent être différents.
        #Aucun des deux ne doit être en gestation, et le male n'est jamais en gestation.
        if sexe_proie != sexe and en_gestation == en_gestation_proie:
            excitation = 0
            excitation_proie = 0
            if sexe:
                nb = nb_lapinous(age, age_proie, fecondite, fecondite_proie)
                jour = temps_gestation_lapin
                en_gestation = True
                gestationnaire = (en_gestation, jour, nb)
                position = (ligne_proie, colonne_proie)
            else:
                nb_proie = nb_lapinous(age, age_proie, fecondite, fecondite_proie)
                jour_proie = temps_gestation_lapin
                en_gestation_proie = True
                gestationnaire_proie = (en_gestation_proie, jour_proie, nb_proie)
                position = (ligne_proie, colonne_proie)
            agents[id_lapin] = ("lapin", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))
            agents[id_proie] = ("lapin", (age_proie, sexe_proie, efficacite_proie, fecondite_proie, excitation_proie, faim_proie, gestationnaire_proie, position_proie))
        environnement[ligne, colonne].remove(id_lapin)
        environnement[ligne_proie, colonne_proie].append(idlapin)
    else:
        #Mouvement aléatoire
        ligne += rd.choice(range(-M,M+1))
        colonne += rd.choice(range(-M,M+1))
        position = (ligne, colonne)
        agents[id_renard] = ("lapin", (age, sexe, efficacite, fecondite, excitation, faim, gestationnaire, position))